# Practical Machine Learning
# Lab 4
## Convolutional Neural Networks

![cnn.jpeg](attachment:cnn.jpeg)
A ConvNet arranges its neurons in three dimensions (width, height, depth), as visualized in one of the layers. Every layer of a ConvNet transforms the 3D input volume to a 3D output volume of neuron activations. In this example, the red input layer holds the image, so its width and height would be the dimensions of the image, and the depth would be 3 (Red, Green, Blue channels)

## Properties of the convolution layer
 - parameter sharing
    - in a fully connected neural network one weight is used only once during a forward step, but in a CNN one weight is used in each position of the input (when the stride is 1, excepting the edges)
 - local connectivity
     - each neuron (unit) is connected only to a small region, this region is named receptive field
 - equivariance to translation
     - if a filter learns to recognise a specific pattern, it will recognise that pattern no matter its location in the input
     
## Properties of the pooling layer
 - a pooling function replaces the output of the net at a certain location with a summary statistic of the nearby outputs
 - helps to make the representation approximately invariant to small translations of the input

In [0]:
import numpy as np
import matplotlib.pyplot as plt 

- Loading the images

In [0]:
# load the training data
train_images_raw = np.load('data/train_images.npy')
train_labels = np.load('data/train_labels.npy') 
print('The number of images for training is %d.' % train_images_raw.shape[0]) 

# load the test data
test_images_raw = np.load('data/test_images.npy')
test_labels = np.load('data/test_labels.npy') 

The number of images for training is 10000.


###  Prepare the data
Convert the data from uint8 to float32, normalize the pixel data from [0, 255] to [0, 1]

Compute the mean image and substract it from each training and test image.

In [0]:
# write your code here  
# TODO: 2.1 convert int to float
train_images = np.float32(train_images_raw) / 255
test_images = np.float32(test_images_raw) / 255
# TODO: 2.2 compute mean training image
mean_image = np.mean(train_images, axis=0)
# TODO: 2.3 subtract the mean image from training and test data.
train_images -= mean_image
test_images -= mean_image  

### Define a conv layer

In [0]:
import tensorflow.keras.layers as layers
layers.Conv2D(filters=64, kernel_size=(5, 5), activation='relu', strides=(1,1), padding='same')
# filters - the number of conv filters
# kernel_size - the size of the kernel, we usually use odd numbers
# activation - the activation function
# strides - the stride on each dimension
# padding - same or valid

### Define a pooling layer

In [0]:
layers.AveragePooling2D(pool_size=3, strides=(2,2), padding='same') # average pooling
layers.MaxPool2D(pool_size=3, strides=(2,2), padding='same') # max pooling 
# pool_size - the size of the window, we usually use odd numbers 
# strides - the stride on each dimension
# padding - same or valid

- Flatten layer - it flattens the input, but does not affect the batch size. 

In [0]:
layers.Flatten()

## Exercises

## 1. Train a convolutional neural network with the following configuration:
    - L1: conv
        - kernel-size=3, stride=1, padding=same, filters=32, activation-function=relu
    - L2: conv
        - kernel-size=3, stride=1, padding=same, filters=32, activation-function=relu
    - L3: pool
        - kernel-size=2, stride=2, padding=same
    - L4: conv
        - kernel-size=3, stride=1, padding=same, filters=64, activation-function=relu
    - L5: conv
        - kernel-size=3, stride=1, padding=same, filters=64, activation-function=relu
    - L6: pool
        - kernel-size=2, stride=2, padding=same
    - L7: conv
        - kernel-size=3, stride=1, padding=same, filters=128, activation-function=relu
    - L8: conv
        - kernel-size=3, stride=1, padding=same, filters=128, activation-function=relu
    - L9: pool
        - kernel-size=2, stride=2, padding=same
    - L10: flatten
    - L11: dropout
        - dropout rate=0.3
    - L12 - fully connected (dense) with 10 units and softmax as activation function.
 
Train the network for 10 epochs using what optimizer you like.

The loss you have to use is 'categorical_crossentropy' (you should encode the training labels using one-hot-encoding in order to use this loss).

Does this network perform better than a fully connected one?
    

In [0]:
# write your code here

# 1.1 encode the training labels in one-hot
labels = np.zeros((len(train_labels), train_labels.max() + 1))
for idx, label in enumerate(train_labels):
    labels[idx, label] = 1 

# 1.2 define the cnn
import tensorflow as tf 

model = tf.keras.models.Sequential([
  layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=(1,1), padding='same'),  
  layers.Conv2D(filters=32, kernel_size=(3, 3), activation='relu', strides=(1,1), padding='same'),  
  layers.MaxPool2D(pool_size=2, strides=(2, 2), padding='same'),
  layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=(1,1), padding='same'),  
  layers.Conv2D(filters=64, kernel_size=(3, 3), activation='relu', strides=(1,1), padding='same'),  
  layers.MaxPool2D(pool_size=2, strides=(2, 2), padding='same'),
  layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=(1,1), padding='same'),  
  layers.Conv2D(filters=128, kernel_size=(3, 3), activation='relu', strides=(1,1), padding='same'),  
  layers.MaxPool2D(pool_size=2, strides=(2, 2), padding='same'),
  layers.Flatten(),
  layers.Dropout(0.3),  
  layers.Dense(10, activation='softmax') 
  ])
# 1.3 compile the cnn
from tensorflow.keras.optimizers import SGD, Adam, RMSprop 

# define the optimizer, set the learning rate and the momentum (for SGD with momentum)
optimizer = Adam(lr=0.001)  
# compile the model specifying the optimizer, the loss and the metrics (as an array).
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['accuracy'])
# 1.4 train the cnn
model.fit(train_images[:-1000], labels[:-1000],
          epochs=10, batch_size=32, initial_epoch=0, 
          validation_data=(train_images[-1000:], labels[-1000:]))  

Train on 9000 samples, validate on 1000 samples
Epoch 1/10
9000/9000 [==============================] - 57s 6ms/sample - loss: 1.8838 - accuracy: 0.2959 - val_loss: 1.5897 - val_accuracy: 0.4080
Epoch 2/10
9000/9000 [==============================] - 57s 6ms/sample - loss: 1.4958 - accuracy: 0.4534 - val_loss: 1.3947 - val_accuracy: 0.4730
Epoch 3/10
9000/9000 [==============================] - 60s 7ms/sample - loss: 1.3203 - accuracy: 0.5248 - val_loss: 1.3472 - val_accuracy: 0.5080
Epoch 4/10
9000/9000 [==============================] - 55s 6ms/sample - loss: 1.1518 - accuracy: 0.5820 - val_loss: 1.1773 - val_accuracy: 0.5900
Epoch 5/10
9000/9000 [==============================] - 55s 6ms/sample - loss: 1.0303 - accuracy: 0.6290 - val_loss: 1.1156 - val_accuracy: 0.6090
Epoch 6/10
9000/9000 [==============================] - 55s 6ms/sample - loss: 0.9044 - accuracy: 0.6783 - val_loss: 1.0242 - val_accuracy: 0.6330
Epoch 7/10
9000/9000 [==============================] - 57s 6ms/sample

In [0]:
t_labels = np.zeros((len(test_labels), test_labels.max() + 1))
for idx, label in enumerate(test_labels):
    t_labels[idx, label] = 1 

test_loss, test_metrics = model.evaluate(test_images, t_labels, verbose=0) 
print('test loss', test_loss)
print('test metrics', test_metrics)

test loss 1.060729025363922
test metrics 0.676


In [0]:
pred = model.predict(test_images)

In [0]:
np.mean(np.argmax(pred, axis=-1) == test_labels)
test loss 1.397190655708313
test metrics 0.645


0.516

In [0]:
i=6
print(labels[i], train_labels[i])

[0. 0. 1. 0. 0. 0. 0. 0. 0. 0.] 2
